In [1]:
import pandas as pd

from helpers import sql

# pandas formatting
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

# import tables
documents = sql("SELECT * FROM csas2_document", database='dmapps_BACKUP')
documenttype = sql("SELECT * FROM csas2_documenttype", database='dmapps_BACKUP')
tracking = sql("SELECT * FROM csas2_documenttracking", database='dmapps_BACKUP')
meetings = sql("SELECT * FROM csas2_meeting", database='dmapps_BACKUP')


In [2]:
# unpublished docs with meetings <= Dec 2020

# Jul #s (approx, backup of db from aug 7)
unpub_2020_aug8 = sql(
    query="""
        SELECT
            csas2_document.id AS document_id,
            csas2_document.status,
            csas2_document.lead_office_id AS region,
            csas2_document.created_at AS document_created,
            csas2_meeting.id AS meeting_id,
            csas2_process.id AS process_id,
            csas2_meeting.start_date AS meeting_date,
            csas2_documenttype.name AS doc_type
        FROM csas2_document
            LEFT JOIN csas2_documenttype ON csas2_document.document_type_id = csas2_documenttype.id
            LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
            LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
        WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
    """, 
    database='dmapps_240807'
)  
unpub_2020_aug8 = unpub_2020_aug8[unpub_2020_aug8['doc_type'] != "Document for translation only (e.g., meeting minutes, terms of reference)"]

# most recent db #s
unpub_2020 = sql(
    query="""
        SELECT
            csas2_document.id AS document_id,
            csas2_document.status,
            csas2_document.lead_office_id AS region,
            csas2_document.created_at AS document_created,
            csas2_meeting.id AS meeting_id,
            csas2_process.id AS process_id,
            csas2_meeting.start_date AS meeting_date,
            csas2_documenttype.name AS doc_type
        FROM csas2_document
            LEFT JOIN csas2_documenttype ON csas2_document.document_type_id = csas2_documenttype.id
            LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
            LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
        WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
    """, 
    database='dmapps_BACKUP'
)  
unpub_2020 = unpub_2020[unpub_2020['doc_type'] != "Document for translation only (e.g., meeting minutes, terms of reference)"]


In [3]:
col_order = ['Science Advisory Report', 'Science Response', 'Proceedings', 'Research Document']
date_columns = ['Aug 8, 2024', 'Dec 30, 2024']

df1 = unpub_2020_aug8.copy()
df1['doc_type'] = pd.Categorical(unpub_2020_aug8['doc_type'], categories=col_order)
df2 = unpub_2020.copy()
df2['doc_type'] = pd.Categorical(unpub_2020['doc_type'], categories=col_order)

print("Outstanding Publication by Type (Meetings December 2020 and Before)")

df = pd.concat(
    [
        pd.DataFrame(df1.drop_duplicates(subset='document_id', keep='first')['doc_type'].value_counts()[col_order]),
        pd.DataFrame(df2.drop_duplicates(subset='document_id', keep='first')['doc_type'].value_counts()[col_order])
    ],
    axis=1
)
df.columns = date_columns
display(df)

print("excluding status = 0")

df = pd.concat(
    [
        pd.DataFrame(df1[df1.status != 0].drop_duplicates(subset='document_id', keep='first')['doc_type'].value_counts().reindex(col_order)),
        pd.DataFrame(df2[df2.status != 0].drop_duplicates(subset='document_id', keep='first')['doc_type'].value_counts().reindex(col_order))
    ],
    axis=1
)
df.columns = date_columns
display(df)


Outstanding Publication by Type (Meetings December 2020 and Before)


,"Aug 8, 2024","Dec 30, 2024"
doc_type,,
Science Advisory Report,17,15
Science Response,0,0
Proceedings,28,13
Research Document,63,53


excluding status = 0


,"Aug 8, 2024","Dec 30, 2024"
doc_type,,
Science Advisory Report,17,15
Science Response,0,0
Proceedings,28,13
Research Document,63,53


# what is different aug to present?

In [4]:
different = set(unpub_2020_aug8.document_id) - set(unpub_2020.document_id)
different
pd.DataFrame(documents[documents.id.isin(different)]['status'].value_counts())

# all look good, either 99 or 12

,count
status,
12,19
99,8


# how many incomplete but with a pub_number?

In [5]:
status = {
   0: "awaiting_changes",
   1: "confirmed",
   2: "submission_date",
   3: "date_chair_sent",
   4: "date_chair_appr",
   5: "date_coordinator_sent",
   6: "date_coordinator_appr",
   13: "date_section_head_sent",
   14: "date_section_head_appr",
   15: "date_division_manager_sent",
   16: "date_division_manager_appr",
   7: "date_director_sent",
   8: "date_director_appr",
   9: "date_doc_submitted",
   10: "date_proof_author_sent",
   11: "date_proof_author_approved",
   12: "actual_posting_date",
   17: "updated_posting_date",
   99: "withdrawn",
}

documents['status_display'] = documents['status'].replace(status)

document_type_id = {
    2: 'Science Advisory Report',
    3: 'Research Document',
    4: 'Proceedings',
    5: 'Science Response',
    6: 'Working Paper',
    8: "Document for translation only (e.g., meeting minutes, terms of reference)",
}

documents['document_type_display'] = documents['document_type_id'].replace(document_type_id)

documents[~documents.status.isin([12, 17, 99]) & documents.pub_number][['id', 'pub_number', 'status', 'status_display', 'document_type_id', 'document_type_display']]

,id,pub_number,status,status_display,document_type_id,document_type_display
50,62,2024/056,9,date_doc_submitted,3,Research Document
123,140,2024/072,9,date_doc_submitted,3,Research Document
143,160,2024/071,6,date_coordinator_appr,3,Research Document
204,223,2025/001,9,date_doc_submitted,3,Research Document
565,668,2023/084,11,date_proof_author_approved,3,Research Document
811,924,2022/006,11,date_proof_author_approved,4,Proceedings
813,926,2023/090,9,date_doc_submitted,3,Research Document
919,1040,2024/028,10,date_proof_author_sent,3,Research Document
928,1050,2024/066,5,date_coordinator_sent,3,Research Document
1005,1131,2025/002,9,date_doc_submitted,3,Research Document


# what about regions?

In [6]:
df1 = unpub_2020_aug8.copy()
df2 = unpub_2020.copy()

regions = {
    1: 'Gulf',
    2: 'Maritimes',
    3: 'Quebec',
    6: 'Newfoundland & Labrador',
    4: 'National',
    7: 'Ontario and Prairie',
    5: 'Pacific',
    8: 'Arctic',
}
df1['region'] = df1['region'].replace(regions)
df1['region'] = pd.Categorical(df1['region'], categories=list(regions.values()))
df2['region'] = df2['region'].replace(regions)
df2['region'] = pd.Categorical(df2['region'], categories=list(regions.values()))

df1 = pd.DataFrame(df1['region'].value_counts().reindex([
    'Arctic',
    'Pacific',
    'Ontario and Prairie',
    'National',
    'Quebec',
    'Gulf',
    'Maritimes',
    'Newfoundland & Labrador',
]))

df2 = pd.DataFrame(df2['region'].value_counts().reindex([
    'Arctic',
    'Pacific',
    'Ontario and Prairie',
    'National',
    'Quebec',
    'Gulf',
    'Maritimes',
    'Newfoundland & Labrador',
]))

df = pd.concat([pd.DataFrame(df1), pd.DataFrame(df2)], axis=1)
df.columns = date_columns
display(df)

,"Aug 8, 2024","Dec 30, 2024"
region,,
Arctic,0,0
Pacific,4,4
Ontario and Prairie,35,27
National,38,38
Quebec,7,7
Gulf,1,1
Maritimes,18,0
Newfoundland & Labrador,10,5


In [7]:
# zero Arctic documents? what about after 2020?
documents[documents.lead_office_id == 8]

# none. ever.

,id,created_at,updated_at,title_en,title_fr,title_iku,pub_number,pages_en,status,old_id,created_by_id,process_id,updated_by_id,url_en,url_fr,dev_link_en,dev_link_fr,ekme_gcdocs_en,ekme_gcdocs_fr,cat_number_en,cat_number_fr,document_type_id,translation_status,pub_number_request_date,due_date,is_confirmed,lead_office_id,pages_fr,pdf_size_kb_en,pdf_size_kb_fr,cat_number_iku,library_link_en,library_link_fr,library_link_iku,pdf_size_kb_iku,isbn_en,isbn_fr,isbn_iku,pages_iku,ekme_gcdocs_iku,pub_number_assigned_date,posting_notification_sent_date,has_data_links,has_third_language,third_language,urgency_notes,urgent,media_attention,sharepoint_archive_en,sharepoint_archive_fr,status_display,document_type_display


# how to group into SUBJECT MATTER categories

In [8]:
branch = sql("SELECT * FROM shared_models_branch")
len(set(branch.name))

34

In [9]:
sector = sql("SELECT * FROM shared_models_sector")
len(set(sector.name))

13

In [10]:
print('SECTOR')
set(sector.name)

SECTOR


{'Aquatic Ecosystems',
 'Atlantic Fisheries Fund',
 'Chief Financial Officer',
 'Ecosystems & Oceans Science',
 'Executive Direction',
 'Fisheries & Harbour Management',
 'Governance Secretariat',
 'Human Resource and Corporate Services',
 'Human Resources and Corporate Services',
 'Innovation and Business Transformation',
 'Internal Audit Directorate',
 'Legal Services Unit',
 'Strategic Policy'}

In [11]:
division = sql("SELECT * FROM shared_models_division")
len(set(division.name))

91

In [12]:
section = sql("SELECT * FROM shared_models_section")
len(set(section.name))

180

In [13]:
region = sql("SELECT * FROM shared_models_region")
len(set(region.name))

8

In [14]:
print('REGION')
set(region.name)

REGION


{'Arctic',
 'Gulf',
 'Maritimes',
 'National',
 'Newfoundland & Labrador',
 'Ontario and Prairie',
 'Pacific',
 'Quebec'}

In [15]:
subjectmatter = sql("SELECT * FROM shared_models_subjectmatter")
len(set(subjectmatter.name))

8

In [16]:
print("SUBJECT MATTER")
set(subjectmatter.name)

SUBJECT MATTER


{'Asian Carp',
 'Groundfish',
 'Herring',
 'Impact Assessment',
 'Major Projects',
 'Marine Mammal',
 'Phytoplankton',
 'eDNA'}

In [17]:
should_be = ['Fisheries', 'Aquatic Ecosystems', 'Marine Mammal', 'Aquaculture']
closest_is = sorted(list(set(sector.name)))
closest_is  # not very close...

['Aquatic Ecosystems',
 'Atlantic Fisheries Fund',
 'Chief Financial Officer',
 'Ecosystems & Oceans Science',
 'Executive Direction',
 'Fisheries & Harbour Management',
 'Governance Secretariat',
 'Human Resource and Corporate Services',
 'Human Resources and Corporate Services',
 'Innovation and Business Transformation',
 'Internal Audit Directorate',
 'Legal Services Unit',
 'Strategic Policy']

### subjectmatter is explicitly only in CSASRequest.tags

In [18]:
request_tags = sql("SELECT * FROM csas2_csasrequest_tags")

In [19]:
# most requests don't have tags...
len(set(request_tags.csasrequest_id))

176

In [20]:
document_tags1 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_subjectmatter.name
    FROM csas2_document
        LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        LEFT JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        LEFT JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        LEFT JOIN csas2_csasrequest_tags ON csas2_csasrequest.id = csas2_csasrequest_tags.csasrequest_id
        LEFT JOIN shared_models_subjectmatter ON csas2_csasrequest_tags.subjectmatter_id = shared_models_subjectmatter.id;
""", database="dmapps_240807") 

df1 = document_tags1['name'].value_counts(dropna=False)

document_tags2 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_subjectmatter.name
    FROM csas2_document
        LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        LEFT JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        LEFT JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        LEFT JOIN csas2_csasrequest_tags ON csas2_csasrequest.id = csas2_csasrequest_tags.csasrequest_id
        LEFT JOIN shared_models_subjectmatter ON csas2_csasrequest_tags.subjectmatter_id = shared_models_subjectmatter.id;
""") 

df2 = document_tags2['name'].value_counts(dropna=False)

df = pd.concat([pd.DataFrame(df1), pd.DataFrame(df2)], axis=1)
df.columns = date_columns
display(df)

,"Aug 8, 2024","Dec 30, 2024"
name,,
None,1152,1200
Marine Mammal,39,39
Groundfish,16,28
Herring,4,7
Major Projects,1,1
Impact Assessment,1,1


In [21]:
document_tags1_2020 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_subjectmatter.name
    FROM csas2_document
        LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        LEFT JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        LEFT JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        LEFT JOIN csas2_csasrequest_tags ON csas2_csasrequest.id = csas2_csasrequest_tags.csasrequest_id
        LEFT JOIN shared_models_subjectmatter ON csas2_csasrequest_tags.subjectmatter_id = shared_models_subjectmatter.id
        LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
            WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
""", database="dmapps_240807") 

df1 = document_tags1_2020['name'].value_counts(dropna=False)

document_tags2_2020 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_subjectmatter.name
    FROM csas2_document
        LEFT JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        LEFT JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        LEFT JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        LEFT JOIN csas2_csasrequest_tags ON csas2_csasrequest.id = csas2_csasrequest_tags.csasrequest_id
        LEFT JOIN shared_models_subjectmatter ON csas2_csasrequest_tags.subjectmatter_id = shared_models_subjectmatter.id
        LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
            WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
""")

df2 = document_tags2_2020['name'].value_counts(dropna=False)

df = pd.concat([pd.DataFrame(df1), pd.DataFrame(df2)], axis=1)
df.columns = date_columns
display(df)

,"Aug 8, 2024","Dec 30, 2024"
name,,
None,114,82
Marine Mammal,1,1


### what if docs were separated into request.sector?

In [22]:
request_sectors = sql("""
    SELECT
        csas2_csasrequest.id AS request_id,
        shared_models_sector.id AS sector_id,
        shared_models_sector.name AS sector_name
    FROM csas2_csasrequest
    JOIN shared_models_section ON shared_models_section.id = csas2_csasrequest.section_id
    JOIN shared_models_division ON shared_models_division.id = shared_models_section.division_id
    JOIN shared_models_branch ON shared_models_branch.id = shared_models_division.branch_id
    JOIN shared_models_sector ON shared_models_sector.id = shared_models_branch.sector_id;
""")

In [23]:
set(request_sectors.sector_name)

{'Aquatic Ecosystems',
 'Ecosystems & Oceans Science',
 'Fisheries & Harbour Management',
 'Strategic Policy'}

In [24]:
df1 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_sector.id AS sector_id,
        shared_models_sector.name AS sector_name
    FROM csas2_document
        JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        JOIN shared_models_section ON shared_models_section.id = csas2_csasrequest.section_id
        JOIN shared_models_division ON shared_models_division.id = shared_models_section.division_id
        JOIN shared_models_branch ON shared_models_branch.id = shared_models_division.branch_id
        JOIN shared_models_sector ON shared_models_sector.id = shared_models_branch.sector_id
""", 
database="dmapps_240807")['sector_name'].value_counts()

df2 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_sector.id AS sector_id,
        shared_models_sector.name AS sector_name
    FROM csas2_document
        JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        JOIN shared_models_section ON shared_models_section.id = csas2_csasrequest.section_id
        JOIN shared_models_division ON shared_models_division.id = shared_models_section.division_id
        JOIN shared_models_branch ON shared_models_branch.id = shared_models_division.branch_id
        JOIN shared_models_sector ON shared_models_sector.id = shared_models_branch.sector_id
""")['sector_name'].value_counts()

df = pd.concat([pd.DataFrame(df1), pd.DataFrame(df2)], axis=1)
df.columns = date_columns
display(df)

,"Aug 8, 2024","Dec 30, 2024"
sector_name,,
Ecosystems & Oceans Science,781,788
Fisheries & Harbour Management,287,337
Aquatic Ecosystems,144,150


In [25]:
df1 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_sector.id AS sector_id,
        shared_models_sector.name AS sector_name
    FROM csas2_document
        JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        JOIN shared_models_section ON shared_models_section.id = csas2_csasrequest.section_id
        JOIN shared_models_division ON shared_models_division.id = shared_models_section.division_id
        JOIN shared_models_branch ON shared_models_branch.id = shared_models_division.branch_id
        JOIN shared_models_sector ON shared_models_sector.id = shared_models_branch.sector_id
        LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
            WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
""", 
database="dmapps_240807")['sector_name'].value_counts()

df2 = sql("""
    SELECT
        csas2_document.id AS document_id,
        csas2_process.id AS process_id,
        csas2_csasrequest.id AS request_id,
        shared_models_sector.id AS sector_id,
        shared_models_sector.name AS sector_name
    FROM csas2_document
        JOIN csas2_process ON csas2_document.process_id = csas2_process.id
        JOIN csas2_process_csas_requests ON csas2_process.id = csas2_process_csas_requests.process_id
        JOIN csas2_csasrequest ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        JOIN shared_models_section ON shared_models_section.id = csas2_csasrequest.section_id
        JOIN shared_models_division ON shared_models_division.id = shared_models_section.division_id
        JOIN shared_models_branch ON shared_models_branch.id = shared_models_division.branch_id
        JOIN shared_models_sector ON shared_models_sector.id = shared_models_branch.sector_id
        LEFT JOIN csas2_meeting ON csas2_process.id = csas2_meeting.process_id
            WHERE DATE(csas2_meeting.start_date) < '2021-01-01'
            AND csas2_document.status NOT IN (12, 17, 99);
""")['sector_name'].value_counts()

df = pd.concat([pd.DataFrame(df1), pd.DataFrame(df2)], axis=1)
df.columns = date_columns
display(df)

,"Aug 8, 2024","Dec 30, 2024"
sector_name,,
Ecosystems & Oceans Science,112,81
Fisheries & Harbour Management,2,1
Aquatic Ecosystems,1,1


### why are there no "Strategic Policy" documents?
(probably because policy questions are not supposed to use the CSAS process)

In [26]:
request_sectors[request_sectors.sector_name == "Strategic Policy"]

,request_id,sector_id,sector_name
1015,329,2,Strategic Policy
1016,356,2,Strategic Policy
1017,402,2,Strategic Policy
1018,466,2,Strategic Policy
1019,1643,2,Strategic Policy


In [27]:
processes = sql("SELECT * FROM csas2_process")
policy_requests = list(request_sectors[request_sectors.sector_name == "Strategic Policy"].request_id)

request_documents = sql("""
    SELECT
        csas2_csasrequest.id AS request_id,
        csas2_csasrequest.status AS request_status,
        csas2_process.id AS process_id,
        csas2_document.id AS document_id
    FROM csas2_csasrequest
        LEFT JOIN csas2_process_csas_requests ON csas2_process_csas_requests.csasrequest_id = csas2_csasrequest.id
        LEFT JOIN csas2_process ON csas2_process_csas_requests.process_id = csas2_process.id
        LEFT JOIN csas2_document ON csas2_document.process_id = csas2_process.id;  
""")
request_documents[request_documents.request_id.isin(policy_requests)]

,request_id,request_status,process_id,document_id
367,329,99,NaN,NaN
408,356,40,NaN,NaN
425,402,40,NaN,NaN
484,466,40,NaN,NaN
2232,1643,99,NaN,NaN
